In [69]:
lib_path = '/Users/ivan/Dropbox/Trading/TA_Parser/lib'
import sys;sys.path.insert(0, lib_path) if lib_path not in sys.path else None
from misc import get_filtered_df, get_history

import pandas as pd
import numpy as np

df = get_filtered_df()
hist = get_history()

In [83]:
def check_results(r: pd.Series) -> pd.Series:
    _max_dtx = 0.5
    _min_profit = 0.6

    r['entered'] = False

    prices = hist[(hist['option'] == r['option'])]
    prices = prices[(prices['date'].dt.date > r['date_time'].date())]
    r['half_date'] = r['date'] + np.timedelta64(int(r['dtx']*_max_dtx), 'D')

    if pd.Timestamp(r['half_date']) >= pd.Timestamp('2023-03-18'):
        return r

    prices = prices[(prices['date']<=r['half_date'])]
    if len(prices)<=2:
        return r

    if prices.iloc[0]['open_interest'] < r['size'] or prices.iloc[0]['ask_low'] > r['price']:
        return r


    close_price = 0
    target_price = r['price'] * (1+_min_profit)
    close_day = None
    for i,pr in prices[1:].iterrows():
        close_price = pr['close_bid']
        close_day = pr['date']

        if pr['open_interest'] < r['size']:
            close_price = pr['open_bid']
            break

        if pr['bid_high'] >= target_price:
            close_price = max(pr['bid_low'], target_price)

    r['entered'] = True
    r['close_day'] = close_day
    r['close_dtx'] = (close_day - r['date']) / np.timedelta64(1, 'D')
    r['close_dtx_pc'] = r['close_dtx'] / r['dtx']
    r['close_price'] = close_price
    r['close_profit'] = close_price - r['price']
    r['close_profit_pc'] = r['close_profit'] / r['price']

    return r

pdf = df[(df['side']>0)]
pdf = pdf.apply(check_results, axis=1)
okdf = pdf[(pdf['entered'])]
okdf

,ask,ask_exch,ask_size,bid,bid_exch,bid_size,check_trade,close_day,close_dtx,close_dtx_pc,...,price,root,side,size,spot,spot_chg,strike,theo,vega_dollar,volume
0,0.10,ARCA,3680.0,0.05,PHLX,8535.0,True,2022-10-07,9.0,0.391304,...,0.10000,SH,2,3680,17.0350,0.035000,19.0,0.078286,3457.0,3681
2,4.60,PHLX,95.0,4.30,PHLX,72.0,True,2022-10-04,6.0,0.260870,...,4.66603,CNI,3,2609,111.3750,2.685000,110.0,4.432300,28552.0,2619
3,1.00,PHLX,37.0,0.85,PHLX,447.0,True,2022-10-07,9.0,0.391304,...,1.00000,SYY,2,2900,73.9000,0.920000,77.0,0.922857,18501.0,2923
5,9.80,PHLX,40.0,9.40,PHLX,54.0,True,2022-10-07,9.0,0.391304,...,9.65000,FIVN,1,4000,77.6550,1.745000,70.0,9.653620,24014.0,4000
7,1.15,PHLX,47.0,0.85,PHLX,70.0,True,2022-11-04,37.0,0.468354,...,1.01001,GDYN,1,834,18.9550,0.825000,22.5,1.007650,2665.0,834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1388,0.75,CBOE,310.0,0.60,CBOE,563.0,True,2023-03-08,8.0,0.470588,...,0.74000,XLB,1,2998,82.0450,0.584982,80.0,0.686404,18122.0,3004
1390,0.55,NOM,25.0,0.45,ARCA,1.0,True,2023-03-08,8.0,0.470588,...,0.55000,CP,2,10331,76.0443,-1.215710,70.0,0.484991,40692.0,21822
1391,0.10,AMEX,1.0,0.00,AMEX,0.0,True,2023-03-08,8.0,0.470588,...,0.10000,CP,2,17218,76.0655,-1.194500,60.0,0.048086,17614.0,20033
1392,0.55,NOM,25.0,0.45,ARCA,1.0,True,2023-03-08,8.0,0.470588,...,0.60000,CP,3,6887,76.0699,-1.190110,70.0,0.481242,27853.0,20709


In [86]:
df

,index,size,option,root,dtx,expiry_date,strike,pc,price,theo,...,hilo,events,bid,ask,open_int,bid_exch,bid_size,ask_size,ask_exch,check_trade
0,1,3680,SH221021C00019000,SH,23,2022-10-21,19.0,C,0.10000,0.078286,...,,,0.05,0.10,1684,PHLX,8535.0,3680.0,ARCA,True
1,61,1438,SAIA221021P00190000,SAIA,23,2022-10-21,190.0,P,7.70000,8.344700,...,,,7.70,10.20,32,AMEX,26.0,17.0,BOX,True
2,70,2609,CNI221021C00110000,CNI,23,2022-10-21,110.0,C,4.66603,4.432300,...,,,4.30,4.60,44,PHLX,72.0,95.0,PHLX,True
3,71,2900,SYY221021C00077000,SYY,23,2022-10-21,77.0,C,1.00000,0.922857,...,,,0.85,1.00,21,PHLX,447.0,37.0,PHLX,True
4,77,2271,CHRW221021C00100000,CHRW,23,2022-10-21,100.0,C,3.49996,3.386130,...,,,3.30,3.50,33,BXO,14.0,21.0,BXO,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1634,15725,2603,BCE230421C00045000,BCE,30,2023-04-21,45.0,C,0.50000,0.586946,...,,,0.50,0.65,487,MPRL,94.0,50.0,PHLX,True
1635,15726,529,HTHT230519C00055000,HTHT,58,2023-05-19,55.0,C,1.60000,1.578639,...,,,1.55,1.60,24,ISE,361.0,17.0,NOM,True
1636,15727,1060,HTHT230519C00055000,HTHT,58,2023-05-19,55.0,C,1.60000,1.573940,...,,,1.55,1.60,24,ISE,336.0,17.0,NOM,True
1637,15733,2001,IWN230421P00095000,IWN,30,2023-04-21,95.0,P,0.30610,0.175172,...,,ExDiv,0.00,0.35,1,BXO,0.0,111.0,PHLX,True


In [85]:
okdf[(okdf['close_profit_pc']<0)]

,ask,ask_exch,ask_size,bid,bid_exch,bid_size,check_trade,close_day,close_dtx,close_dtx_pc,...,price,root,side,size,spot,spot_chg,strike,theo,vega_dollar,volume
0,0.10,ARCA,3680.0,0.05,PHLX,8535.0,True,2022-10-07,9.0,0.391304,...,0.10000,SH,2,3680,17.0350,0.035000,19.0,0.078286,3457.0,3681
3,1.00,PHLX,37.0,0.85,PHLX,447.0,True,2022-10-07,9.0,0.391304,...,1.00000,SYY,2,2900,73.9000,0.920000,77.0,0.922857,18501.0,2923
7,1.15,PHLX,47.0,0.85,PHLX,70.0,True,2022-11-04,37.0,0.468354,...,1.01001,GDYN,1,834,18.9550,0.825000,22.5,1.007650,2665.0,834
11,1.00,BZX,20.0,0.65,BZX,20.0,True,2022-10-07,9.0,0.391304,...,1.00826,BFH,3,587,32.4300,0.580000,35.0,0.835711,1751.0,604
12,1.05,BZX,171.0,0.70,PHLX,12.0,True,2022-10-07,9.0,0.391304,...,1.05000,BFH,2,689,32.4200,0.570000,35.0,0.880688,2066.0,1704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,0.70,NOM,1.0,0.55,AMEX,129.0,True,2023-03-08,8.0,0.470588,...,0.70000,SFM,2,4200,30.6050,-0.325000,29.0,0.627415,9593.0,7089
1388,0.75,CBOE,310.0,0.60,CBOE,563.0,True,2023-03-08,8.0,0.470588,...,0.74000,XLB,1,2998,82.0450,0.584982,80.0,0.686404,18122.0,3004
1390,0.55,NOM,25.0,0.45,ARCA,1.0,True,2023-03-08,8.0,0.470588,...,0.55000,CP,2,10331,76.0443,-1.215710,70.0,0.484991,40692.0,21822
1391,0.10,AMEX,1.0,0.00,AMEX,0.0,True,2023-03-08,8.0,0.470588,...,0.10000,CP,2,17218,76.0655,-1.194500,60.0,0.048086,17614.0,20033
